In [1]:
import numpy as np
from src.utils.picard import SORsolver

N = 32
a = np.random.randn(N,N)
a += 10 * np.ones_like(a)
print(a.shape)

b = np.random.randn(N)
print(b.shape)

eps = 1e-8
w = 1.4

conv = eps
is_print = True

x, loss_list = SORsolver(a,b,w,eps,3,conv,is_print)
loss_list

(32, 32)
(32,)
steps : 3 / 3, loss : 0.503, not converged


[0.12993953773333414, 0.25631921801510016, 0.5032826474999778]

In [2]:
p = a - np.diag(a.diagonal())
l1 = p[0,0:10] 
l2 = np.ones(l1.shape[0])
print(l1)
print(l2)
l1@l2

[ 0.          9.89843772 11.79615754  9.41915316 10.80554586 10.29858061
  9.69556977  8.70527256  9.10776494  9.21597437]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


88.94245652419862

In [3]:
A = np.array([[4, -1, -6, 0],
              [-5, -4, 10, 8],
              [0, 9, 4, -2],
              [1, 0, -7, 5]])

B = np.array([2, 21, -12, -6])

w = 0.8

x, loss_list = SORsolver(A,B,w,eps,64,conv,is_print)
print(x)

steps : 64 / 64, loss : 763699586.533, not converged
[-1427933437210649600  1989994704671686656  -388988091880160128
  -347639771048461440]


/home/zinzinbin/codes_for_study/fusion-plasma/plasma-shape-control/src/utils/picard.py:23: RuntimeWarning: overflow encountered in long_scalars
  sig = A[idx,0:idx]@x_new[0:idx] + A[idx,idx+1:]@x[idx+1:]


In [3]:
import numpy as np

def sor_solver(A, b, omega, initial_guess, convergence_criteria):
    """
    This is an implementation of the pseudo-code provided in the Wikipedia article.
    Arguments:
        A: nxn numpy matrix.
        b: n dimensional numpy vector.
        omega: relaxation factor.
        initial_guess: An initial solution guess for the solver to start with.
        convergence_criteria: The maximum discrepancy acceptable to regard the current solution as fitting.
    Returns:
        phi: solution vector of dimension n.
    """
    step = 0
    phi = initial_guess[:]
    residual = np.linalg.norm(np.matmul(A, phi) - b)  # Initial residual
    while residual > convergence_criteria:
        for i in range(A.shape[0]):
            sigma = 0
            for j in range(A.shape[1]):
                if j != i:
                    sigma += A[i, j] * phi[j]
            phi[i] = (1 - omega) * phi[i] + (omega / A[i, i]) * (b[i] - sigma)
        residual = np.linalg.norm(np.matmul(A, phi) - b)
        step += 1
        print("Step {} Residual: {:10.6g}".format(step, residual))
    return phi

# An example case that mirrors the one in the Wikipedia article
residual_convergence = 1e-8
omega = 0.5  # Relaxation factor

A = np.array([[4, -1, -6, 0],
              [-5, -4, 10, 8],
              [0, 9, 4, -2],
              [1, 0, -7, 5]])

b = np.array([2, 21, -12, -6])

initial_guess = np.zeros(4)

phi = sor_solver(A, b, omega, initial_guess, residual_convergence)
print(phi)

Step 1 Residual:    14.6179
Step 2 Residual:    11.2948
Step 3 Residual:    3.67768
Step 4 Residual:    3.19424
Step 5 Residual:    2.53585
Step 6 Residual:    0.68396
Step 7 Residual:    0.75634
Step 8 Residual:   0.593797
Step 9 Residual:   0.189146
Step 10 Residual:   0.190827
Step 11 Residual:   0.144984
Step 12 Residual:  0.0584829
Step 13 Residual:  0.0500644
Step 14 Residual:  0.0366401
Step 15 Residual:  0.0175857
Step 16 Residual:  0.0134192
Step 17 Residual: 0.00950158
Step 18 Residual:  0.0051081
Step 19 Residual: 0.00363697
Step 20 Residual: 0.00250919
Step 21 Residual: 0.00144923
Step 22 Residual: 0.000991173
Step 23 Residual: 0.000670862
Step 24 Residual: 0.000405216
Step 25 Residual: 0.000270828
Step 26 Residual: 0.000180835
Step 27 Residual: 0.000112307
Step 28 Residual: 7.40858e-05
Step 29 Residual: 4.90057e-05
Step 30 Residual: 3.09616e-05
Step 31 Residual: 2.0275e-05
Step 32 Residual: 1.3326e-05
Step 33 Residual: 8.50875e-06
Step 34 Residual: 5.54917e-06
Step 35 Resi

In [9]:
A = np.random.randn(128,128)
B = np.random.randn(128)
A += 10 * np.ones_like(A)

omega = 1.2
initial_guess = np.zeros(128)
phi = sor_solver(A, B, omega, initial_guess, residual_convergence)
print(phi)

Step 1 Residual:    192.077
Step 2 Residual:    1918.19
Step 3 Residual:    24570.1
Step 4 Residual:     302604
Step 5 Residual: 3.52061e+06
Step 6 Residual: 3.68034e+07
Step 7 Residual: 3.23503e+08
Step 8 Residual: 2.48489e+09
Step 9 Residual: 2.07009e+10
Step 10 Residual: 2.21205e+11
Step 11 Residual: 2.65818e+12
Step 12 Residual: 3.13257e+13
Step 13 Residual: 3.26827e+14
Step 14 Residual: 2.86729e+15
Step 15 Residual: 2.20566e+16
Step 16 Residual: 2.24222e+17
Step 17 Residual: 3.36975e+18
Step 18 Residual: 4.78273e+19
Step 19 Residual:  5.892e+20
Step 20 Residual: 6.34302e+21
Step 21 Residual: 5.98193e+22
Step 22 Residual: 4.91819e+23
Step 23 Residual: 3.66324e+24
Step 24 Residual: 3.39156e+25
Step 25 Residual: 4.57148e+26
Step 26 Residual: 5.98483e+27
Step 27 Residual: 6.73662e+28
Step 28 Residual: 6.45932e+29
Step 29 Residual: 5.22276e+30
Step 30 Residual: 3.89094e+31
Step 31 Residual: 4.58488e+32
Step 32 Residual: 7.16381e+33
Step 33 Residual: 9.77717e+34
Step 34 Residual: 1.1418

/tmp/ipykernel_19340/2143514725.py:23: RuntimeWarning: overflow encountered in double_scalars
  sigma += A[i, j] * phi[j]
/tmp/ipykernel_19340/2143514725.py:23: RuntimeWarning: invalid value encountered in double_scalars
  sigma += A[i, j] * phi[j]
/tmp/ipykernel_19340/2143514725.py:25: RuntimeWarning: overflow encountered in matmul
  residual = np.linalg.norm(np.matmul(A, phi) - b)
/tmp/ipykernel_19340/2143514725.py:25: RuntimeWarning: invalid value encountered in matmul
  residual = np.linalg.norm(np.matmul(A, phi) - b)
